<a href="https://colab.research.google.com/github/ekqlsrla/ESAA-2/blob/main/HW/1121_CH9_8_Surprise_Package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 머신러닝 완벽 가이드
---

## **| 08** 파이썬 추천 시스템 패키지 - Surprise

### 1. Surprise 패키지 소개

* 다양한 추천 알고리즘 예를 들어 **사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++,NMF 기반의 잠재 요인 협업 필터링**을 쉽게 적용해 추천 시스템 구축


In [31]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. Surprise를 이용한 추천 시스템 구축

1) Surprise는 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트를 가져오는 API 제공

In [32]:
#관련 모듈 임포트
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [33]:
! kubectl apply -f https://k8s.io/examples/admin/dns/dnsutils.yaml

/bin/bash: kubectl: command not found


In [34]:
from surprise import Reader
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/ml-100k/ml-100k.csv')
reader = Reader(rating_scale=(1, 5))
col = ['user_id', 'item_id', 'rating']
df = df[col]
data = Dataset.load_from_df(df, reader)

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
#data = Dataset.load_builtin('ml-100k',prompt = False)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

* SVD로 잠재 요인 협업 필터링 수행

: `algo=SVD()`와 같이 알고리즘 객체를 생성한 뒤 **fit**을 수행해 학습 데이터 세트 기반으로 추천 알고리즘 학습

In [37]:
algo = SVD()
algo.fit(trainset)

* `test()` : 사용자 - 아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
* `predict()` : 개별 사용자와 영화에 대한 추천 평점 반환
  * `was_impossible = True` : 예측값을 **생성할 수 없는** 데이터

In [38]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), 'size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'> size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid=120, iid=282, r_ui=4.0, est=3.465220104709705, details={'was_impossible': False}),
 Prediction(uid=882, iid=291, r_ui=4.0, est=3.762593277682684, details={'was_impossible': False}),
 Prediction(uid=535, iid=507, r_ui=5.0, est=3.8765756531683713, details={'was_impossible': False}),
 Prediction(uid=697, iid=244, r_ui=5.0, est=3.4729283559095037, details={'was_impossible': False}),
 Prediction(uid=751, iid=385, r_ui=4.0, est=3.2859432969398616, details={'was_impossible': False})]

In [20]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[(120, 282, 3.6980617006844536),
 (882, 291, 3.8965454355692124),
 (535, 507, 4.141170157190634)]

In [39]:
#predict 메서드 사용

uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.53   {'was_impossible': False}


In [40]:
#추천 예측 평점과 실제 평점과의 차이 평가

accuracy.rmse(predictions)

RMSE: 0.9474


0.9473582678253816

### 3. Surprise 주요 모듈 소개

#### (1) Dataset

|API 명|내용|
|---|---|
|`Dataset.load_builtin(name = 'ml-100k')`|무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 받기|
|`Dataset.load_from_file(file_path,reader)`|OS 파일에서 데이터를 로딩할 때 사용|
|`Dataset.load_from_df(df,reader)`|판다스의 DataFrame에서 데이터를 로딩|

#### (2) OS 파일 데이터를 Surprise 데이터 세트를 로딩

In [41]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings.csv')
ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings_noh.csv',index = False, header = False)

In [42]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp',sep = ',', rating_scale = (0.5,5))
data = Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings_noh.csv',reader = reader)

* `line_format(string)` : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
* `sep(char)` : 칼럼을 분리하는 분리자이며, 디폴트는 **'\t'**
* `rating_scale(tuple,optional)` : 평점 값의 **최소~최대** 평점 설정

In [43]:
trainset,testset = train_test_split(data,test_size = 0.25, random_state = 0)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### (3) 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [45]:
ratings.head()

,1,1.1,4.0,964982703
0,1,3,4.0,964981247
1,1,6,4.0,964982224
2,1,47,5.0,964983815
3,1,50,5.0,964982931
4,1,70,3.0,964982400


In [47]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings.csv')
reader = Reader(rating_scale = (0.5,5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset = train_test_split(data,test_size = 0.25,random_state = 0)

algo = SVD(n_factors = 50,random_state = 0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 4. Surprise 추천 알고리즘 클래스

|클래스명|설명|
|---|---|
|SVD|행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘|
|KNNBasic|최근접 이웃 협업 필터링을 위한 KNN알고리즘|
|BaselineOnly|사용자 Biase와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘|

* SVD 클래스의 입력 파라미터
  * `n_factors` : 잠재 요인 K의 개수
  * `n_epochs` : SGD 수행시 반복 횟수
  * `biased (bool)` : 베이스라인 사용자 편향 적용 여부

### 5. 베이스라인 평점

: 개인의 성향을 반영해 아이템 평가에 **편향성 요소**를 반영하여 평점을 부과하는 것

* 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
* 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
* 아이템 편향 점수 = 아이테별 평점 평균 값 - 전체 평균 평점

### 6. 교차 검증과 하이퍼 파라미터 튜닝

In [49]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings.csv')
reader = Reader(rating_scale = (0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state = 0)
cross_validate(algo, data, measures = ['RMSE','MAE'],cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8680  0.8776  0.8780  0.8672  0.8724  0.8726  0.0046  
MAE (testset)     0.6690  0.6731  0.6750  0.6681  0.6705  0.6711  0.0026  
Fit time          5.84    5.59    5.69    7.02    6.28    6.09    0.53    
Test time         0.16    0.16    0.16    0.28    0.16    0.18    0.05    


{'test_rmse': array([0.86802611, 0.87759922, 0.87797268, 0.8671591 , 0.87239463]),
 'test_mae': array([0.66895452, 0.67314717, 0.67495158, 0.66805645, 0.6704517 ]),
 'fit_time': (5.843271255493164,
  5.5880303382873535,
  5.694894313812256,
  7.024224519729614,
  6.284754753112793),
 'test_time': (0.16070222854614258,
  0.15598249435424805,
  0.15866470336914062,
  0.2816586494445801,
  0.15772676467895508)}

In [56]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

#CV : 3개 폴트세트 / 성능 평가 : rmse , mse
gs = GridSearchCV(SVD, param_grid, measures = ['rmse','mae'],cv = 3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])



0.8770924716607708
{'n_epochs': 20, 'n_factors': 50}


### 7. Surprise를 이용한 개인화 영화 추천 시스템 구축

* **Surprise** 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 적절한 영화 추천하기

In [59]:
#train_test_split으로 분리되지 않은 데이터 세트에 fit()을 호출해 오류가 발생

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: ignored

* **데이터 세트 전체**를 학습 데이터로 사용하려면 `DatasetAutoFolds`클래스를 이용

In [60]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale =(0.5,5))
data_folds = DatasetAutoFolds(ratings_file = '/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/ratings_noh.csv',reader = reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [61]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

* 특정 사용자는 **userId = 9** 인 사용자로 지정

In [63]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA-2/DATA/MovieLens Latest/movies.csv')

#userID = 9의 movieId 데이터를 추출해 movieID = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] ==9]['movieId']

if movieIds[movieIds==42].count() == 0 :
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [64]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid,iid,verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [65]:
def get_unseen_surprise(ratings, movies,userId) :
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
  total_movies = movies['movieId'].tolist()
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수 : ', len(seen_movies), '추천 대상 영화 수 :', len(unseen_movies), '전체 영화 수 :', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies,9)

평점 매긴 영화 수 :  46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742


In [66]:
def recomm_movie_by_surprise(algo,userId, unseen_movies, top_n = 10) :
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의
  def sortkey_est(pred) :
    return pred.est

  predictions.sort(key = sortkey_est, reverse = True)
  top_predictions = predictions[:top_n]

  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in zip (top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)

print('##### Top - 10 추천 영화 리스트 #####')
for top_movie in top_movie_preds :
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수 :  46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742
##### Top - 10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
